## Importing Libraries

In [86]:
import requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import time
import concurrent.futures
import modules.psql as psql

## Postgres Configuration

In [87]:
%run config_psql.ipynb

## Setting Configurations

In [88]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Getting match list and urls from database

In [89]:
query = """
SELECT sub3.match_id, sub3.url
FROM
(
	SELECT sub2.match_id, eu.url, ROW_NUMBER() OVER (PARTITION BY sub2.match_id) AS rk
	FROM (
		SELECT m.match_id
		FROM dwh.match m
		JOIN (
			SELECT DISTINCT match_id 
			FROM dwh.match_player AS mp
			WHERE NOT EXISTS (
				SELECT 1 
				FROM dwh.match_player 
				WHERE match_id = mp.match_id 
				AND (is_captain OR is_wicketkeeper)
			)
		) sub1
		ON m.match_id = sub1.match_id	
	) sub2
	LEFT JOIN dwh.espn_url eu ON sub2.match_id = eu.match_id AND eu.url_type = 'match'
) sub3
WHERE sub3.rk <= 1
"""

In [90]:
with engine.connect() as conn:
    df = pd.read_sql_query(query,con = engine)

## Initializing Parameters

In [91]:
url_list = list(map(lambda url: url.replace(url.split("/")[-1], "match-playing-xi"), df['url']))
df_result = pd.DataFrame(columns=['match_id','team','wicket_keeper','captain'])

In [92]:
def find_wk_c(url):
    response = requests.get(url)
    if response.status_code == 404:
        print("Url not found")
        return None
        
    soup = BeautifulSoup(response.content, 'lxml')
    team = {
        'match_id' : url.split('/')[-2].split('-')[-1],
        '1_name' : soup.find('table').find('thead').find_all('th')[1].text,
        '2_name' : soup.find('table').find('thead').find_all('th')[2].text
    }
    for row in soup.find('table').find('tbody').find_all('tr'):
        cols = len(row.find_all('td'))
        a_cols = len(row.find_all('a'))+1
        if cols != 1:
            for num in range(1,a_cols):
                player_title = row.find_all('a')[num-1]['title']
                if ((chr(8224) in player_title) or ('(WK)' in player_title) or ('(wk)' in player_title)):
                    team[str(num) + '_wicket_keeper'] = row.find_all('a')[num-1]['href'].split('/')[-1].split('-')[-1]
                if (('(C)' in player_title) or ('(c)' in player_title) or '(captain)' in player_title):
                    team[str(num) + '_captain'] = row.find_all('a')[num-1]['href'].split('/')[-1].split('-')[-1]
        else:
            break
    return team

In [93]:
num_threads = 50
num_series = len(df)

with concurrent.futures.ThreadPoolExecutor(max_workers = num_threads) as executor:
    futures = []
    for url in url_list:
        futures.append(executor.submit(find_wk_c, url))

    for future in (concurrent.futures.as_completed(futures)):
        result = future.result()
        #team 1 record
        record = {
            'match_id': result['match_id'],
            'team': result['1_name'],
            'wicket_keeper': result['1_wicket_keeper'],
            'captain': result['1_captain']
        }
        df_result.loc[len(df_result)] = record
        #team 2 record
        record = {
            'match_id': result['match_id'],
            'team': result['2_name'],
            'wicket_keeper': result['2_wicket_keeper'],
            'captain': result['2_captain']
        }            
        df_result.loc[len(df_result)] = record

In [94]:
df_result.head(1000)

,match_id,team,wicket_keeper,captain
0,1373581,West Indies,604302,820351
1,1373581,England,308967,308967
2,1375871,Australia Women,275486,275486
3,1375871,South Africa Women,499254,922481
4,1375870,Australia Women,275486,275486
5,1375870,South Africa Women,499254,922481
6,1375869,Australia Women,275486,275486
7,1375869,South Africa Women,499254,922481
8,1375846,Australia,326434,489889
9,1375846,West Indies,1168667,348024


In [95]:
# Load match information into Stage table
with engine.connect() as conn:
    conn.execute("TRUNCATE TABLE stg.match_wk_c")

count_rows = df_result.to_sql('match_wk_c', schema = 'stg', con = engine, if_exists='append', method = 'multi', index = False)

In [96]:
df_result[df_result['match_id'] == '1409517']

,match_id,team,wicket_keeper,captain
376,1409517,Otago Women,592594,54565
377,1409517,Canterbury Women,598155,312174


In [ ]:
"""
UPDATE dwh.match_player mp
SET is_wicketkeeper = true
FROM (
    SELECT stg.match_id, p.id, p.identifier, p.unique_name
    FROM stg.match_wk_c stg
    JOIN dwh.people p ON (stg.wicket_keeper = p.key_cricinfo OR stg.wicket_keeper = p.key_cricinfo_2)
) AS sub
WHERE mp.match_id = sub.match_id AND mp.player_id_num = sub.id;
"""